In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import os
import keras
import time
from os import makedirs
from keras.preprocessing.image import save_img
import sys 
from matplotlib import pyplot
from keras.models import Sequential 
from keras.layers import Conv2D 
from keras.layers import MaxPooling2D
from keras.layers import Dense 
from keras.layers import Flatten 
from keras.layers import  Dropout
from keras.optimizers import SGD 
from sklearn.model_selection import KFold
from numpy import mean
from numpy import std
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib.pyplot import plot
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import ModelCheckpoint 
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
import numpy as np
starttime = time.time()
classlist = ['airplane','automobile','bird','cat','deer']
label_names = np.array(('airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck'))

In [ ]:
#GetTrainingData i.e data the network would be trained with.
def getTrainingData(train_data_normalized,train_labels,label_names,batchcount):
    traindatacount = batchcount   
    classlist = ['airplane','automobile','bird','cat','deer']
    classcounterdict = {'airplane':0,'automobile':0,'bird':0,'cat':0,'deer':0}
    newtraindata, newtrainlabels,newfilenames = [], [],[]
    counter = 1
    while counter < 50000:
        label = label_names[train_labels[counter]][0]        
        img_array = train_data_normalized[counter,:,:,:]
        if label in classlist:          
            if classcounterdict[label] + 1 <= traindatacount:
                newtraindata.append(img_array)
                newtrainlabels.append(train_labels[counter])               
                classcounterdict[label] = classcounterdict[label] + 1
        counter = counter + 1
    newtraindata = np.array(newtraindata)   
    newtrainlabels = np.array(newtrainlabels)
    return newtraindata,newtrainlabels

In [ ]:
#get test data ,this data is used for evaluation after training is performed 
def getTestData(label_names,test_data_train_data_normalized,test_labels,batchcount):
    testdatacount = batchcount  
    classlist = ['airplane','automobile','bird','cat','deer']
    classcounterdict = {"airplane":0,"automobile":0,"bird":0,"cat":0,"deer":0}
    newtestdata, newtestlabels,newtestfilenames = [], [],[]
    counter = 1
    while counter < 10000:
        label = label_names[test_labels[counter]][0]
        img_array = test_data_train_data_normalized[counter,:,:,:]
        if label in classlist:
            if classcounterdict[label] + 1 <= testdatacount:            
                newtestdata.append(img_array)            
                newtestlabels.append(test_labels[counter])             
                classcounterdict[label] = classcounterdict[label] + 1
        counter = counter + 1
    newtestdata = np.array(newtestdata)   
    newtestlabels = np.array(newtestlabels)
    return newtestdata,newtestlabels

In [ ]:
#function to normalize both train and test data 
def  prep_pixels(train,test):
     train_norm = train.astype('float32')
     test_norm = test.astype('float32')
     train_norm = train_norm / 255 
     test_norm = test_norm / 255
     return  train_norm , test_norm 


In [ ]:
#This a VGG model architecture
def define_model(dropoutval,learningRate,momentumvalue,epochs):    
     model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dropoutval)) 
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dropoutval)) 
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dropoutval))
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dropoutval))
    model.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
    model.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
    model.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
    model.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')) 
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(dropoutval))
    model.add(Dense(1024, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(512, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(5, activation='softmax'))
    decay_rate = learningRate / epochs 
    sgd = SGD(lr=learningRate, momentum=momentumvalue, decay=decay_rate, nesterov=False) 
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# function to evaluate network after training is done 
def evaluate_model_new(dataX, dataY,testdataX,testdataY,batchsize,epochsize,model):
    scores, histories = list(), list()
    filepath="weights.best.hdf5" 
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max') 
    callbacks_list = [checkpoint]
    datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1,horizontal_flip=True, vertical_flip=True) 
    it_train = datagen.flow(dataX, dataY, batch_size=batchsize) 
    steps = int(dataX.shape[0] / batchsize)   
    history = model.fit_generator(it_train, steps_per_epoch=steps, epochs=epochsize, validation_data=(testdataX, testdataY), verbose=0,callbacks=callbacks_list) 
    _, acc = model.evaluate(testdataX, testdataY, verbose=0)
    print('> %.3f' % (acc * 100.0))    
    histories.append(history)
    scores.append(acc)
    return scores, histories


In [ ]:
#Plot Accuracy and Loss curve 
def summarize_diagnostics(histories):
    for i in range(len(histories)):
        # plot loss
        pyplot.subplot(211)
        pyplot.title('Cross Entropy Loss')
        pyplot.plot(histories[i].history['loss'], color='blue', label='train')
        pyplot.plot(histories[i].history['val_loss'], color='orange', label='test')
        # plot accuracy
        pyplot.subplot(212)
        pyplot.title('Classification Accuracy')
        pyplot.plot(histories[i].history['acc'], color='blue', label='train')
        pyplot.plot(histories[i].history['val_acc'], color='orange', label='test')
        pyplot.show()

In [ ]:
#function to calculate accuracy
def summarize_performance(scores):
    # print summary
    print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
    # box and whisker plots of results
    pyplot.boxplot(scores)
    pyplot.show()

In [ ]:
#training and evaluation is performed here , we call all functions here .
from keras.utils import to_categorical
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
train_data_normalized,test_data_train_data_normalized = prep_pixels(x_train,x_test)
trainingDataRecordsPerClassCount = 5000
testDataRecordsPerClassCount = 1000
batchsize = 32
epochsize = 120
dropoutval = 0.2
learningRate = 0.01
momentumvalue = 0.9
newtraindata,newtrainlabels = getTrainingData(train_data_normalized,y_train,label_names,trainingDataRecordsPerClassCount)
newtestdata,newtestlabels = getTestData(label_names,test_data_train_data_normalized,y_test,testDataRecordsPerClassCount) 
newtrainlabels = to_categorical(newtrainlabels)
newtestlabels =  to_categorical(newtestlabels)
model=define_model(dropoutval,learningRate,momentumvalue,epochsize)
scores, histories = evaluate_model_new(newtraindata, newtrainlabels,newtestdata,newtestlabels,batchsize,epochsize,model)
# learning curves
summarize_diagnostics(histories)
summary_performances(scores)

